In [1]:
import pandas as pd
import regex as re
from collections import Counter
import json
import time
import requests
import collections
import matplotlib.pyplot as plt
import numpy as np
import pprint
import pycountry 
import pycountry_convert as pc
import os
import pandas as pd
import nltk
import re

In [ ]:
# may want to integrate the google scrape when selecting patents? because for some it doesn't find anything. ZA? TW? 

In [ ]:
# still need to sort out why do 5 patents disappear in the final step?

In [2]:
with open(R'X:\5_Research\Paul\data\genes_of_technology\full_patent_data_lens_EP_test_marine_3319.json') as json_string:
    data = json.load(json_string)

In [1]:
#Load API key
my_key = open(R"C:\\Users\\pauld\\OneDrive\\PhD\\Code\\keys\\thelens.txt", "r").read()

In [10]:
# simple query for all family members of one patent
import requests
url = 'https://api.lens.org/patent/search'
request_body = '''{
                      "query": {
                                "terms": {
                                            "family.simple.member.lens_id": ["001-712-126-350-412"]
                                            }
                                }          
                    }'''
headers = {'Authorization': my_key, 'Content-Type': 'application/json'}
response = requests.post(url, data=request_body, headers=headers)
if response.status_code != requests.codes.ok:
      print(response.status_code)
else:
      print(response.text)
data_single = response.json()



{"total":56,"max_score":1.0,"data":[{"lens_id":"001-712-126-350-412","jurisdiction":"EP","doc_number":"1911837","kind":"A2","date_published":"2008-04-16","doc_key":"EP_1911837_A2_20080416","docdb_id":289466762,"lang":"en","biblio":{"publication_reference":{"jurisdiction":"EP","doc_number":"1911837","kind":"A2","date":"2008-04-16"},"application_reference":{"jurisdiction":"EP","doc_number":"07019867","kind":"A","date":"2001-09-28"},"priority_claims":{"claims":[{"jurisdiction":"EP","doc_number":"01985723","kind":"A","date":"2001-09-28","sequence":1},{"jurisdiction":"US","doc_number":"23630300","kind":"P","date":"2000-09-28","sequence":2},{"jurisdiction":"US","doc_number":"29756201","kind":"P","date":"2001-06-12","sequence":3}],"earliest_claim":{"date":"2000-09-28"}},"invention_title":[{"text":"FAD5-2, Mitglied der Desaturase-Familie und Verwendungen davon","lang":"de"},{"text":"FAD5-2 fatty acid desaturase family member and uses thereof","lang":"en"},{"text":"FAD5-2, membre de la famille 

In [3]:
# simple query for all family members of several patents
patent_ids = []
lens_ids = []

for scroll in data:
    for k,v in scroll.items():
        if k=='data': 
            for patent in v: 
                #print(patent)
                patent_ids.append(re.sub(r'_[^_]*$', '', patent['doc_key']).replace('_',''))
                lens_ids.append(patent['lens_id'])
df = pd.DataFrame({'patent_id':patent_ids, 'lens_id': lens_ids})

In [18]:
df

,patent_id,lens_id
0,US9410134B2,000-959-609-762-989
1,JP2003532698A,001-815-591-912-275
2,US9422558B2,003-170-265-500-408
3,US8420312B2,004-053-513-540-817
4,JP2018511602A,004-582-080-858-734
...,...,...
3314,JP2011509655A,195-314-182-864-954
3315,JP2014087338A,196-392-806-052-39X
3316,US9493529B2,196-858-690-413-529
3317,JP2002519385A,198-054-492-211-028


In [64]:
# make list of patentIDs (got to include the double quotes); delete duplicates
data_list = json.dumps(list(set(df['lens_id'])))

len(list(set(df['lens_id'])))

3319

In [65]:
# simple query for all family members 

start_time = time.time()  
data_all = []
url = 'https://api.lens.org/patent/search'
# request body with scroll time of 1 minute
request_body = '''{
  "query": {
      "terms":  {
          "family.simple.member.lens_id": %s
      }
  }
  , "size": 100
  , "scroll": "1m"
}''' % (data_list)
headers = {'Authorization': my_key, 'Content-Type': 'application/json'}

  
# Recursive function to scroll through paginated results
def scroll(scroll_id):
  # Change the request_body to prepare for next scroll api call
  # Make sure to append the include fields to make faster response
    
    if scroll_id is not None:        
        global request_body
        request_body = '''{"scroll_id": "%s"}''' % (scroll_id)
    
    # make api request
    response = requests.post(url, data=request_body, headers=headers) 
    
    if response.status_code != 204:
        if response.status_code == requests.codes.too_many_requests:
            time.sleep(8)
            print('TOOO MANY AGGGH')
            scroll(scroll_id)

        # If the response is ok, do something with the response, take the new scroll id and iterate
        
        # leave the function if it returns empty (because then everything is downloaded already)
        else:
            print('next scroll')
            json = response.json()
            #print(response.json())
            
            if 'scroll_id' in json:
                scroll_id = json['scroll_id']
            else:
                print('No scroll_id found in response.')
                return # Extract the new scroll id from response
            print('Response Status Code:', response.status_code)
            data_all.append(response.json())
            print("--- %s seconds ---" % (time.time() - start_time))
            scroll(scroll_id)
    else: 
        print('empty response')
        #print('eey finished')
        
# start recursive scrolling
scroll(scroll_id=None)

next scroll
Response Status Code: 200
--- 43.7769136428833 seconds ---
next scroll
Response Status Code: 200
--- 48.52700591087341 seconds ---
next scroll
Response Status Code: 200
--- 58.44443869590759 seconds ---
next scroll
Response Status Code: 200
--- 67.29380822181702 seconds ---
next scroll
Response Status Code: 200
--- 74.0151035785675 seconds ---
next scroll
Response Status Code: 200
--- 79.04955911636353 seconds ---
next scroll
Response Status Code: 200
--- 84.89945650100708 seconds ---
next scroll
Response Status Code: 200
--- 91.28555130958557 seconds ---
next scroll
Response Status Code: 200
--- 97.26312899589539 seconds ---
next scroll
Response Status Code: 200
--- 101.51965737342834 seconds ---
next scroll
Response Status Code: 200
--- 106.28866720199585 seconds ---
next scroll
Response Status Code: 200
--- 111.03341269493103 seconds ---
next scroll
Response Status Code: 200
--- 116.78795194625854 seconds ---
next scroll
Response Status Code: 200
--- 121.06245684623718 s

next scroll
Response Status Code: 200
--- 676.1096339225769 seconds ---
TOOO MANY AGGGH
next scroll
Response Status Code: 200
--- 689.5516610145569 seconds ---
next scroll
Response Status Code: 200
--- 695.4710648059845 seconds ---
next scroll
Response Status Code: 200
--- 700.7055222988129 seconds ---
next scroll
Response Status Code: 200
--- 710.2688422203064 seconds ---
next scroll
Response Status Code: 200
--- 721.0917880535126 seconds ---
next scroll
Response Status Code: 200
--- 727.4042196273804 seconds ---
next scroll
Response Status Code: 200
--- 733.0406038761139 seconds ---
next scroll
Response Status Code: 200
--- 738.5957560539246 seconds ---
next scroll
Response Status Code: 200
--- 744.2402720451355 seconds ---
next scroll
Response Status Code: 200
--- 749.6814813613892 seconds ---
next scroll
Response Status Code: 200
--- 764.5750594139099 seconds ---
next scroll
Response Status Code: 200
--- 771.702507019043 seconds ---
next scroll
Response Status Code: 200
--- 777.588

next scroll
Response Status Code: 200
--- 1360.4194812774658 seconds ---
next scroll
Response Status Code: 200
--- 1365.0919661521912 seconds ---
next scroll
Response Status Code: 200
--- 1369.810141324997 seconds ---
next scroll
Response Status Code: 200
--- 1375.4201724529266 seconds ---
next scroll
Response Status Code: 200
--- 1380.295684337616 seconds ---
next scroll
Response Status Code: 200
--- 1385.4491856098175 seconds ---
next scroll
Response Status Code: 200
--- 1392.7474284172058 seconds ---
next scroll
Response Status Code: 200
--- 1397.5920615196228 seconds ---
next scroll
Response Status Code: 200
--- 1403.3320291042328 seconds ---
next scroll
Response Status Code: 200
--- 1408.857057094574 seconds ---
TOOO MANY AGGGH
next scroll
Response Status Code: 200
--- 1422.9989721775055 seconds ---
next scroll
Response Status Code: 200
--- 1428.73570895195 seconds ---
next scroll
Response Status Code: 200
--- 1433.919665813446 seconds ---
next scroll
Response Status Code: 200
---

In [66]:
# save it
with open(R'X:\5_Research\Paul\data\genes_of_technology\full_patent_data_lens_marine_3319_all_families_unedited.json', 'w') as f:
    json.dump(data_all, f)


In [3]:
# open it
with open(R'X:\5_Research\Paul\data\genes_of_technology\full_patent_data_lens_marine_3319_all_families_unedited.json') as json_string:
    data_all = json.load(json_string)


In [ ]:
# need to filter by original 
df_old = pd.read_excel(R'X:\5_Research\Paul\data\genes_of_technology\full_patent_data_lens_marine_3319.xlsx', index_col = 0)

patents_select = []
patent_count = 0
for scroll in data_all:
    for k,v in scroll.items():
        if k=='data': 
            for patent in v:
                if patent['lens_id'] in list(df_old['lens_id']): 
                    patent_count = patent_count+1
                    print(patent_count)
                    nr = 1
                    family_ids = []
                    family_tuples = []
                    print(patent['lens_id'])

                    #create a list of family members for that patent

                    for keyss, valuess in patent['families']['simple_family'].items(): 
                        if keyss == 'members':
                            #print(keyss)
                            for document in valuess: 
                                #print(nr)
                                #print(valuess)
                                family_ids.append(document['lens_id'])

                                nr = nr +1


                    #create a list of tuples of family members
                    for scroll in data_all:
                        for ke,va in scroll.items():
                            if ke=='data': 
                                for patent_f in va: 
                                    if patent_f['lens_id'] in family_ids: 
                                        #save patent id, patent date, patent type
                                        family_tuples.append((patent_f['lens_id'], patent_f['publication_type'], patent_f['date_published'], patent['lens_id'])) # also adding patent['lens_id'] to get the original patent id to merge it back to the dataframe and keep everything traceable

                    ##select only the first application per family, but overwrite with the first granted patent of the family, if that exists

                    earliest_granted_patent = None
                    earliest_patent_application = None

                    for family_tuple in family_tuples:
                        if family_tuple[1] == 'GRANTED_PATENT':
                            if earliest_granted_patent is None or family_tuple[2] < earliest_granted_patent[2]:
                                earliest_granted_patent = family_tuple
                        elif family_tuple[1] == 'PATENT_APPLICATION':
                            if earliest_patent_application is None or family_tuple[2] < earliest_patent_application[2]:
                                earliest_patent_application = family_tuple

                    if earliest_granted_patent is not None:
                        earliest = earliest_granted_patent
                    else:
                        earliest = earliest_patent_application
                    patents_select.append(earliest)

                    #if patent_count == 3:  
                    #    break
    #break


In [ ]:
#delete duplicates
def remove_duplicates(lst):
    # Create an empty dictionary to store tuples as keys
    encountered = {}
    # Create an empty list to store unique tuples
    result = []
    # Iterate through the input list
    for tup in lst:
        match_tup = tup[:3]
        # If the tuple has not been encountered before,
        # add it to the dictionary and the result list
        if match_tup not in encountered:
            encountered[match_tup] = True
            result.append(tup)
    # Return the list of unique tuples
    return result
#Driver code
patents_select_nodupl = remove_duplicates(patents_select)
#This code is contributed by Edula Vinay Kumar Reddy

In [ ]:
len(patents_select_nodupl)

In [ ]:
#how many applications and granted patents are there? 
granted = []
for tupl in patents_select_nodupl:
    #print(tupl)
    if 'GRANTED_PATENT' in tupl: 
        granted.append(tupl)
    

len(granted)


In [ ]:
# get the patent_ids from the selected ones, so that I can curate their Google patent links and scrape them

# actually the new DF should have all available info, not just from the tuples.. 

df_tuples = pd.DataFrame(patents_select_nodupl, columns = ['lens_id_new', 'publication_type', 'date', 'lens_id'])

patent_ids = []
doc_keys = []
lens_ids = []
dates = []
families = []
codes = []
abstracts = []
claims = []
legal_status = []
publication_type = []

for scroll in data_all:
    for k,v in scroll.items():
        if k=='data': 
            for patent in v: 
                #print(patent)
                if patent['lens_id'] in list(df_tuples['lens_id_new']) :
                    patent_ids.append(re.sub(r'_[^_]*$', '', patent['doc_key']).replace('_',''))
                    lens_ids.append(patent['lens_id'])
                    doc_keys.append(patent['doc_key'])
                    dates.append(patent['date_published'])
                    families.append(patent['families'])
                    if 'classifications_ipcr' in patent['biblio'].keys():  
                        if 'classifications' in patent['biblio']['classifications_ipcr']:
                            codes.append([x['symbol'] for x in patent['biblio']['classifications_ipcr']['classifications']])
                    else: 
                        codes.append(np.nan)
                    if 'abstract' in patent.keys(): 
                        abstracts.append(patent['abstract'])
                    else: 
                        abstracts.append(np.nan)
                    if 'claims' in patent.keys(): 
                        claims.append(patent['claims'])
                    else: 
                        claims.append(np.nan)

                    legal_status.append(patent['legal_status'])
                    publication_type.append(patent['publication_type'])
                
                
                
df_new = pd.DataFrame(list(zip(patent_ids,doc_keys,lens_ids,dates, families, codes, abstracts, claims, legal_status, publication_type)), columns = ['patent_id','doc_key','lens_id_new','date_published', 'families', 'codes', 'abstract', 'claims', 'legal_status', 'publication_type'])

In [ ]:
df_new

In [ ]:
df_tuples

In [ ]:
# merge with tuple to get the old lens patent id
df_new_witholdid = df_new.merge(df_tuples, on = 'lens_id_new')

In [ ]:
# merge to get organisms and patent url (if available) from old patent id
df = df_new_witholdid.merge(df_old[['lens_id','organism','patent_url']].dropna(subset = ['lens_id']), on = 'lens_id', how = 'left').rename(columns = {'lens_id': 'lens_id_old', 'patent_url':'patent_url_old','publication_type_x': 'publication_type'}).drop(columns = ['date', 'publication_type_y']) 

In [ ]:
#create new URLs for edited patents
nr = 0
patent_url_new = []
for new_id in list(df['lens_id_new']):
    if new_id != list(df['lens_id_old'])[nr]:
        patent_url_new.append('https://patents.google.com/patent/'+ str(list(df['patent_id'])[nr])+'/en')
        
    else: 
        patent_url_new.append(list(df['patent_url_old'])[nr])
    nr = nr+1

df['patent_url_new'] = patent_url_new

df

In [ ]:
df.to_csv(R'X:\5_Research\Paul\data\genes_of_technology\full_patent_data_lens_marine_2689_familychecked.csv')

## Trash: check for diversity of codes in families

In [ ]:
# how is this structured? 
family_check = []
nr = 1
checklist = []
for scroll in data_all:
    for k,v in scroll.items():
        if k=='data': 
            for patent in v: 
                print(patent['lens_id'])
                for keys, values in patent.items():
                    for keyss, valuess in patent['families']['simple_family'].items(): 
                        print(keyss)
                        for document in valuess: 
                            print(document)
                            family_check.append(document['lens_id'])
                            print(nr)
                            nr = nr +1
                        
                        break
                    break
                break
    break
                
                #nr = nr +1
                #print(nr)
    
        #        print(patent['doc_key'])
        

In [ ]:
# so do I now have data for all of these? because the family ids I already had before... 

In [6]:
family_check

['153-308-850-885-941',
 '138-959-970-155-201',
 '126-489-645-286-479',
 '181-914-600-112-191',
 '079-453-983-704-808',
 '071-763-769-291-38X',
 '189-120-247-892-831',
 '093-197-692-306-807',
 '153-863-155-936-017',
 '030-627-324-459-350',
 '019-093-431-741-958',
 '183-493-125-877-709',
 '012-055-828-698-554',
 '065-872-365-945-645',
 '113-940-356-121-952',
 '136-127-444-894-018',
 '000-090-604-949-076',
 '022-503-895-401-043',
 '065-520-996-936-324',
 '197-855-731-198-494',
 '096-538-295-602-014',
 '161-690-346-306-189',
 '103-841-431-720-079']

In [63]:
# let's see.. 
# how is this structured? 

nr = 1
for scroll in data_all:
    for k,v in scroll.items():
        if k=='data': 
            for patent in v: 
                if patent['lens_id'] in family_check: 
                    print(patent)
                    print('OOOOHH next /n/n')
                
                
    
                
                #nr = nr +1
                #print(nr)
    
        #        print(patent['doc_key'])
        

{'lens_id': '000-736-028-898-716', 'jurisdiction': 'JP', 'doc_number': '2010501531', 'kind': 'A', 'date_published': '2010-01-21', 'doc_key': 'JP_2010501531_A_20100121', 'docdb_id': 317757786, 'biblio': {'publication_reference': {'jurisdiction': 'JP', 'doc_number': '2010501531', 'kind': 'A', 'date': '2010-01-21'}, 'application_reference': {'jurisdiction': 'JP', 'doc_number': '2009525079', 'kind': 'A', 'date': '2007-08-24'}, 'priority_claims': {'claims': [{'jurisdiction': 'EP', 'doc_number': '06119557', 'kind': 'A', 'date': '2006-08-25', 'sequence': 1}, {'jurisdiction': 'US', 'doc_number': '84060006', 'kind': 'P', 'date': '2006-08-28', 'sequence': 2}, {'jurisdiction': 'EP', 'doc_number': '2007058830', 'kind': 'W', 'date': '2007-08-24', 'sequence': 3}], 'earliest_claim': {'date': '2006-08-25'}}, 'parties': None, 'classifications_ipcr': {'classifications': [{'symbol': 'A61K38/00'}, {'symbol': 'A61P19/02'}, {'symbol': 'A61P19/04'}, {'symbol': 'C07K14/50'}]}, 'classifications_cpc': {'classif

In [ ]:
# yesss! so now I just need to format it in a way that is useful

In [30]:
# so I guess just seeing how the IPC codes change within family?

# let's see.. 
# how is this structured? 
codes = []
nr = 1
for scroll in data_all:
    for k,v in scroll.items():
        if k=='data': 
            for patent in v: 
                if patent['lens_id'] in family_check: 
                    print(patent['lens_id'])
                    #print(patent['doc_number'])
                    #print(patent['kind'])
                    print([x['symbol'] for x in patent['biblio']['classifications_ipcr']['classifications']])
                    codes.append([x['symbol'] for x in patent['biblio']['classifications_ipcr']['classifications']])
                    
                    #print('OOOOHH next /n/n')
                
                
    
                
                #nr = nr +1
                #print(nr)
    
        #        print(patent['doc_key'])
        

000-090-604-949-076
['C12Q1/22', 'C12Q1/48']
065-520-996-936-324
['C12Q1/48', 'C12Q1/22']
153-863-155-936-017
['C12Q1/22', 'C12Q1/48']
079-453-983-704-808
['C12Q1/70', 'C12Q1/22', 'C12Q1/48']
183-493-125-877-709
['C12Q/', 'C12Q1/22', 'C12Q1/48']
030-627-324-459-350
['C12Q/', 'C12Q1/22', 'C12Q1/48']
103-841-431-720-079
['C12Q1/22', 'C12Q1/48']
093-197-692-306-807
['C12Q1/22', 'C12Q1/48']
153-308-850-885-941
['C12Q1/22', 'C12Q1/48']
126-489-645-286-479
['C12Q1/22', 'C12Q1/48']
181-914-600-112-191
['C12Q1/70', 'C12Q1/22', 'C12Q1/48']
161-690-346-306-189
['C12Q1/70', 'C12Q1/22', 'C12Q1/48']
012-055-828-698-554
['C12Q1/48', 'C12Q1/22']
189-120-247-892-831
['C12Q1/22', 'C12Q1/48']
197-855-731-198-494
['C12Q1/48', 'C12N9/12', 'C12Q1/22']
138-959-970-155-201
['C12Q1/48', 'C12N9/12', 'C12Q1/22']
096-538-295-602-014
['C12Q1/48', 'C12N15/09', 'C12Q1/22', 'G01N21/76']
022-503-895-401-043
['C12Q1/48', 'C12Q1/22']
019-093-431-741-958
['C12Q1/22', 'C12Q1/48']
071-763-769-291-38X
['C12Q1/48', 'C12Q1/2

In [29]:
count = Counter(codes)
count.most_common()

TypeError: unhashable type: 'list'

In [28]:
codes

[['C12Q1/22', 'C12Q1/48'],
 ['C12Q1/48', 'C12Q1/22'],
 ['C12Q1/22', 'C12Q1/48'],
 ['C12Q1/70', 'C12Q1/22', 'C12Q1/48'],
 ['C12Q/', 'C12Q1/22', 'C12Q1/48'],
 ['C12Q/', 'C12Q1/22', 'C12Q1/48'],
 ['C12Q1/22', 'C12Q1/48'],
 ['C12Q1/22', 'C12Q1/48'],
 ['C12Q1/22', 'C12Q1/48'],
 ['C12Q1/22', 'C12Q1/48'],
 ['C12Q1/70', 'C12Q1/22', 'C12Q1/48'],
 ['C12Q1/70', 'C12Q1/22', 'C12Q1/48'],
 ['C12Q1/48', 'C12Q1/22'],
 ['C12Q1/22', 'C12Q1/48'],
 ['C12Q1/48', 'C12N9/12', 'C12Q1/22'],
 ['C12Q1/48', 'C12N9/12', 'C12Q1/22'],
 ['C12Q1/48', 'C12N15/09', 'C12Q1/22', 'G01N21/76'],
 ['C12Q1/48', 'C12Q1/22'],
 ['C12Q1/22', 'C12Q1/48'],
 ['C12Q1/48', 'C12Q1/22'],
 ['C12Q1/22', 'C12Q1/48'],
 ['C12Q1/48', 'C12N15/09', 'C12Q1/00', 'C12Q1/02', 'C12Q1/22', 'G01N21/76'],
 ['C12Q1/22', 'C12Q1/48']]

1) create empty family list and data dictionary 
2) go into patent, check if family is already in list --> if yes, skip
3) if not, create new family id : list of patent_id family members pair and add to dictionary
4) iterate again through existing data to add data of each family member to the dictionary

In [ ]:
# how is this structured? 

nr = 1
checklist = []
for scroll in data_all:
    for k,v in scroll.items():
        if k=='data': 
            for patent in v: 
                print(patent['lens_id'])
                for keys, values in patent.items():
                    for keyss, valuess in patent['families']['simple_family'].items(): 
                        print(keyss)
                        for document in valuess: 
                            print(document)
                            print(nr)
                            nr = nr +1
                        
                        break
                    break
                break
    break
                
                #nr = nr +1
                #print(nr)
    
        #        print(patent['doc_key'])
        

In [23]:
# how is this structured? 

nr = 0
checklist = []
for scroll in data_all:
    for k,v in scroll.items():
        if k=='data': 
            for patent in v: 
                print(patent['lens_id'])
                if patent['lens_id'] in list(set(df['lens_id']))[:10]:
                    nr = nr +1
                    print(nr)
        #        print(patent['doc_key'])
    

000-736-028-898-716
001-426-451-347-463
001-971-051-463-186
004-775-782-617-004
005-957-393-650-824
007-604-566-876-821
010-255-552-421-442
014-225-471-747-588
016-449-696-589-731
016-813-802-104-418
019-546-989-080-692
019-947-095-313-597
020-257-893-293-479
021-422-655-929-809
023-432-517-517-242
024-828-503-655-617
026-454-184-610-794
026-943-142-875-10X
029-084-391-500-970
029-429-002-013-441
030-679-633-435-007
030-854-523-866-776
036-970-916-088-430
037-085-078-364-751
043-243-789-945-227
043-464-414-980-332
047-355-182-878-429
049-446-665-149-657
054-385-379-342-418
057-184-877-961-826
057-266-103-713-684
058-929-265-048-745
061-625-319-982-679
062-794-147-146-896
066-841-722-319-242
067-733-443-217-612
069-601-491-342-790
070-105-675-521-075
070-284-797-085-466
071-040-284-568-070
074-028-407-020-468
074-269-125-773-979
074-525-512-200-273
074-966-618-982-721
076-540-799-788-85X
1
078-519-405-400-67X
079-971-374-577-591
081-199-655-982-703
081-884-739-537-75X
083-198-845-736-10

In [ ]:
# simple query for family members
import requests
url = 'https://api.lens.org/patent/search'
request_body = '''{
  "query": {
        "bool": {
            "must": [
                {
                    "match" : {
                        "legal_status.granted": true
                    }
                },
                {
                    "term" : {
                        "publication_type": "GRANTED_PATENT"
                    }
                },
                {
                    "terms": {
                        "family.simple.member.lens_id": ["001-712-126-350-412"]
                    }
                }
            ]
        }
    }          
}'''
headers = {'Authorization': my_key, 'Content-Type': 'application/json'}
response = requests.post(url, data=request_body, headers=headers)
if response.status_code != requests.codes.ok:
      print(response.status_code)
else:
      print(response.text)

In [ ]:
# simple query for family members
import requests
url = 'https://api.lens.org/patent/search'
data = '''{
              "query": {
                  "terms":  {
                      "family.simple.member.lens_id": ["001-712-126-350-412"]
                  }
              },
              "include": ["biblio", "doc_key"]
}'''
headers = {'Authorization': my_key, 'Content-Type': 'application/json'}
response = requests.post(url, data=data, headers=headers)
if response.status_code != requests.codes.ok:
      print(response.status_code)
else:
      print(response.text)

# simple query for family members
import requests
url = 'https://api.lens.org/patent/search'
request_body = '''{
  "query": {
        "bool": {
            "must": [
                {
                    "match" : {
                        "legal_status.granted": true
                    }
                },
                {
                    "term" : {
                        "publication_type": "GRANTED_PATENT"
                    }
                },
                {
                    "terms": {
                        "family.simple.member.lens_id": ["001-712-126-350-412"]
                    }
                }
            ]
        }
    }          
}'''
headers = {'Authorization': my_key, 'Content-Type': 'application/json'}
response = requests.post(url, data=request_body, headers=headers)
if response.status_code != requests.codes.ok:
      print(response.status_code)
else:
      print(response.text)

# simple query for family members
import requests
url = 'https://api.lens.org/patent/search'
request_body = '''{
    "query": {
        "bool": {
            "must": [
                {
                    "match" : {
                        "legal_status.granted": true
                    }
                },
                {
                    "term" : {
                        "publication_type": "PATENT_APPLICATION"
                    }
                },
                {
                    "term" : {
                        "jurisdiction": "US"
                    }
                },
                {
                    "range": {
                        "year_published": {
                            "gte": 2022
                        }
                    }
                }
            ]
        }
    }
}'''
headers = {'Authorization': my_key, 'Content-Type': 'application/json'}
response = requests.post(url, data=request_body, headers=headers)
if response.status_code != requests.codes.ok:
      print(response.status_code)
else:
      print(response.text)

# simple query for family members

data_granted = []
url = 'https://api.lens.org/patent/search'
# request body with scroll time of 1 minute
request_body = '''{
  "query": {
      "terms":  {
          "family.simple.member.lens_id": %s
      }
  }
  , "size": 100
  , "scroll": "1m"
}''' % (data_list_test)
headers = {'Authorization': my_key, 'Content-Type': 'application/json'}

  
# Recursive function to scroll through paginated results
def scroll(scroll_id):
  # Change the request_body to prepare for next scroll api call
  # Make sure to append the include fields to make faster response
    
    if scroll_id is not None:        
        global request_body
        request_body = '''{"scroll_id": "%s"}''' % (scroll_id)
        
    # make api request
    response = requests.post(url, data=request_body, headers=headers) 
    
    # If rate-limited, wait for n seconds and proceed the same scroll id
    # Since scroll time is 1 minutes, it will give sufficient time to wait and proceed
    if response.status_code == requests.codes.too_many_requests:
        time.sleep(8)
        scroll(scroll_id)
    # If the response is not ok here, better to stop here and debug it
    
    elif response.status_code != requests.codes.ok:
        print(response.json())
        print('Äahh')
    # If the response is ok, do something with the response, take the new scroll id and iterate
    
    # leave the function if it returns empty (because then everything is downloaded already)
    json = response.json()
    if json['data'] == []:
        return
    
    else:
        print('next scroll')
        json = response.json()
        scroll_id = json['scroll_id'] # Extract the new scroll id from response
        #print(json['data']) #DO something with your data
        data_granted.append(response.json())
        
        scroll(scroll_id)
        
# start recursive scrolling
scroll(scroll_id=None)


data_granted

# search for linked granted patents
data_granted = []
url = 'https://api.lens.org/patent/search'
# request body with scroll time of 1 minute
request_body = '''{
  "query": {
        "bool": {
            "must": [
                {
                    "match" : {
                        "legal_status.granted": true
                    }
                },
                {
                    "term" : {
                        "publication_type": "GRANTED_PATENT"
                    }
                },
                {
                    "term": {
                        "family.simple.member.lens_id": %s
                    }
                }
            ]
        }
    }
}''' % (data_list_test)
headers = {'Authorization': my_key, 'Content-Type': 'application/json'}


# Recursive function to scroll through paginated results
def scroll(scroll_id):
  # Change the request_body to prepare for next scroll api call
  # Make sure to append the include fields to make faster response
    
    if scroll_id is not None:        
        global request_body
        request_body = '''{"scroll_id": "%s"}''' % (scroll_id)
        
    # make api request
    response = requests.post(url, data=request_body, headers=headers) 
    
    # If rate-limited, wait for n seconds and proceed the same scroll id
    # Since scroll time is 1 minutes, it will give sufficient time to wait and proceed
    if response.status_code == requests.codes.too_many_requests:
        time.sleep(8)
        scroll(scroll_id)
    # If the response is not ok here, better to stop here and debug it
    
    elif response.status_code != requests.codes.ok:
        print(response.json())
        print('Äahh')
    # If the response is ok, do something with the response, take the new scroll id and iterate
    
    # leave the function if it returns empty (because then everything is downloaded already)
    json = response.json()
    if json['data'] == []:
        return
    
    else:
        print('next scroll')
        json = response.json()
        scroll_id = json['scroll_id'] # Extract the new scroll id from response
        #print(json['data']) #DO something with your data
        data_granted.append(response.json())
        
        scroll(scroll_id)
        
# start recursive scrolling
scroll(scroll_id=None)
